In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout
)
from tensorflow.keras.optimizers import Adam


In [2]:
train_dir = "../data/split/train"
val_dir   = "../data/split/val"
test_dir  = "../data/split/test"

IMG_SIZE = 224


In [3]:
def load_data(base_dir):
    X = []
    y = []
    
    for label, cls in enumerate(["Normal", "TB"]):
        class_dir = os.path.join(base_dir, cls)
        
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                continue
            
            img = img / 255.0
            img = np.expand_dims(img, axis=-1)
            
            X.append(img)
            y.append(label)
    
    return np.array(X), np.array(y)


In [4]:
X_train, y_train = load_data(train_dir)
X_val, y_val     = load_data(val_dir)
X_test, y_test   = load_data(test_dir)

X_train.shape, y_train.shape


((2940, 224, 224, 1), (2940,))

In [5]:
print("Train TB:", np.sum(y_train == 1))
print("Train Normal:", np.sum(y_train == 0))


Train TB: 490
Train Normal: 2450


In [7]:
from tensorflow.keras.layers import Input

model = Sequential([
    Input(shape=(224, 224, 1)),

    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])


In [8]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16
)


ValueError: You must call `compile()` before using the model.

In [9]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [10]:
model.optimizer


In [11]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16
)


Epoch 1/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.8694 - loss: 0.3158 - val_accuracy: 0.9222 - val_loss: 0.1928
Epoch 2/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.9293 - loss: 0.1775 - val_accuracy: 0.9508 - val_loss: 0.1469
Epoch 3/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - accuracy: 0.9320 - loss: 0.1518 - val_accuracy: 0.9571 - val_loss: 0.1225
Epoch 4/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - accuracy: 0.9514 - loss: 0.1304 - val_accuracy: 0.9667 - val_loss: 0.1077
Epoch 5/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.9565 - loss: 0.1121 - val_accuracy: 0.9714 - val_loss: 0.0989
Epoch 6/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.9629 - loss: 0.0959 - val_accuracy: 0.9667 - val_loss: 0.0949
Epoch 7/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - accuracy: 0.9687 - loss: 0.0847 - val_accuracy: 0.9540 - val_loss: 0.1231
Epoch 8/10
184/184 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9707 - loss: 0.0800 - val_accu

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 491ms/step - accuracy: 0.9794 - loss: 0.0713
Test Loss: 0.07125826925039291
Test Accuracy: 0.9793650507926941


In [13]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)


20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 491ms/step


In [14]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm


array([[520,   5],
       [  8,  97]])

In [15]:
TN, FP, FN, TP = cm.ravel()

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Sensitivity (TB recall):", sensitivity)
print("Specificity (Normal recall):", specificity)


Sensitivity (TB recall): 0.9238095238095239
Specificity (Normal recall): 0.9904761904761905


In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=["Normal", "TB"]))


              precision    recall  f1-score   support

      Normal       0.98      0.99      0.99       525
          TB       0.95      0.92      0.94       105

    accuracy                           0.98       630
   macro avg       0.97      0.96      0.96       630
weighted avg       0.98      0.98      0.98       630



In [17]:
model.save("../models/tb_cnn_model.h5")


In [18]:
model.save("../models/tb_cnn_model.keras")
